## 05-MACAU 

In this notebook I use MACAU - Mixed model Association for Count data via data AUgmentation - to assess the influence of Olympia oyster size/weight on DNA methylation, while controlling for relatedness. I leverage sequence data from bisulfite treated DNA from 2 Olympia oyster populations - Dabob Bay (in Hood Canal), and Oyster Bay (in South Puget Sound). 

MACAU runs on Lenox. The software is available on [Zhou Lab website](http://www.xzlab.org/software.html). The R program seems to have some issues, so I'm using the binary version installed by Sam on Roadrunner. The [MACAU user manual](http://www.xzlab.org/software/macau/MACAUmanual.pdf) is the basis for this analysis. 

### Input files 

MACAU inputs for this type of analysis include: 

(1) Methylated read counts (raw counts, not percentages)
(2) Total read counts (raw counts)
(3) Relatedness matrix
(4) Predictor variable, or covariates 

First, I download all the files I'll need for MACAU. The files were generated as follows: 

Both read count files - (1) and (2) above - were generated by Laura Spencer in RStudio in the [04-raw-count-files.Rmd notebook](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/code/04-raw-count-files.Rmd), using the MethylKit object that Steven Roberts generated in MethylKit in the [01-methylkit.Rmd notebook](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/code/01-methylkit.Rmd). Note that in both files, counts from + and - strand were combined. These matricese were created from a MethylKit object that had been filtered for all loci with minimum of 10x coverage and maximum of 100x coverage, and with loci that were present in 8 of the 9 samples per population.

- [counts-filtered-total.txt](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/analyses/counts-filtered-total.txt?raw=true)
- [counts-filtered-meth.txt](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/analyses/counts-filtered-meth.txt?raw=true)

The relatedness matrix - (3) - was generated by Katherine Silliman from 2bRad data, using SNPs and the program ANGSD. Check out her notebook entry, [SOS_angsd.ipynb](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/analyses/2bRAD/SOS_angsd.ipynb). 
- [HCSSmbdsamples_rab.txt](https://github.com/sr320/paper-oly-mbdbs-gen/blob/3f81bc8fdb8bc02f2a8aa585613fbc1b27b33333/analyses/2bRAD/HCSSmbdsamples_rab.txt) - generated using only HC/SS samples  
- [mbdsamples_rab.txt](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/analyses/2bRAD/mbdsamples_rab.txt) - using all three populations - HC/SS/NF  

The predictor covariates - (4) - is the variable we will use to assess differential DNA methylation. Our data is both shell length and wet weight. 
- [size.macau.txt](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/data/size.macau.txt)

NOTE: all 4 files need to have samples in the same order. They are all ordered using the mbd seq. sample numbers, 1-18. For the count data files, they automatically were ordered sequentially this way when I generated them. For the size covariate file, I re-orded samples in the [04-raw-count-files.Rmd notebook](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/code/04-raw-count-files.Rmd).  Katherine re-ordered her relatedness matrix in the [SOS_angsd.ipynb notebook](https://github.com/sr320/paper-oly-mbdbs-gen/blob/master/analyses/2bRAD/SOS_angsd.ipynb). 

Here are screen shots to confirm. 

_Hold for SSs of other dataframes from RStudio_

Katherine's relatedness matrix order 
<img src="attachment:image.png" width="600">

See if we can access MACAU 

In [21]:
! /home/shared/macau/macau -h


 MACAU version 1.00alpha, released on 06/05/2015
 implemented by Xiang Zhou

 type ./macau -h [num] for detailed helps
 options: 
 1: quick guide
 2: file I/O related
 3: fit binomial mixed models
 4: note



In [24]:
! /home/shared/macau/macau -h 2

 FILE I/O RELATED OPTIONS
 -g        [filename]      specify input read count file name
          format: site id, individual ID#1, individual ID#2, individual ID#3, ...
                  siteID#1, expression for individual 1, expression for individual 2, expression for individual 3, ...
                  siteID#2, expression for individual 1, expression for individual 2, expression for individual 3,  ...
                  ...
          missing expression value: NA
 -t        [filename]      specify input total read count file (optional); in the same format as above, or contain only one data row
 -p        [filename]      specify input: predictor variable file name
          format: variable for individual 1
                  variable for individual 2
                  ...
          missing predictor variable value: NA
 -n        [filename]      specify column number for predictor variable file (optional)
 -k        [filename]      specify input kinship/relatedness matrix file name
 -v

In [4]:
pwd

u'/home/srlab/GitHub/paper-oly-mbdbs-gen/code'

In [4]:
# create a macau/ directory  

! mkdir ../analyses/macau/

In [5]:
# MACAU writes output files to current directory, so move there 
%cd ../analyses/macau/

/home/srlab/GitHub/paper-oly-mbdbs-gen/analyses/macau


In [6]:
pwd

u'/home/srlab/GitHub/paper-oly-mbdbs-gen/analyses/macau'

In [28]:
# Check out methylated count file
! head ../../analyses/counts-filtered-meth.txt

siteID	numCs1	numCs2	numCs3	numCs4	numCs5	numCs6	numCs7	numCs8	numCs9	numCs10	numCs11	numCs12	numCs13	numCs14	numCs15	numCs16	numCs17	numCs18
Contig0_39226	11	17	10	14	9	13	9	10	18	9	4	15	10	5	8	8	8	8
Contig0_39234	10	7	10	5	12	11	7	8	8	7	1	10	8	5	12	5	12	10
Contig0_71523	16	12	13	16	11	8	11	10	15	19	17	13	24	18	28	18	17	18
Contig0_71533	21	17	17	17	21	13	17	20	20	27	22	17	32	23	33	22	23	22
Contig0_71542	21	22	16	14	15	12	28	13	18	21	19	12	35	25	32	21	16	21
Contig0_71546	20	19	14	12	12	15	24	13	15	19	15	11	30	21	26	20	15	16
Contig0_71558	20	18	12	15	13	5	25	11	18	28	21	19	29	25	29	25	23	18
Contig0_71617	18	13	21	17	14	8	20	19	12	20	18	12	23	24	29	17	23	17
Contig0_71647	19	19	14	21	15	9	21	28	19	19	26	15	20	27	33	29	28	20


In [29]:
# Check out total count file
! head ../../analyses/counts-filtered-total.txt

siteID	coverage1	coverage2	coverage3	coverage4	coverage5	coverage6	coverage7	coverage8	coverage9	coverage10	coverage11	coverage12	coverage13	coverage14	coverage15	coverage16	coverage17	coverage18
Contig0_39226	21	22	22	23	23	21	20	16	31	17	5	23	17	10	20	13	19	24
Contig0_39234	24	27	27	26	28	25	23	24	36	19	7	29	19	12	25	17	23	32
Contig0_71523	16	12	13	18	13	8	16	10	16	23	18	14	28	21	33	21	20	18
Contig0_71533	21	17	18	17	21	13	23	20	21	27	22	21	33	24	36	23	23	23
Contig0_71542	23	23	16	14	15	12	28	14	18	21	19	12	35	26	33	23	17	22
Contig0_71546	20	19	15	12	14	15	26	14	16	21	15	11	31	22	30	21	15	17
Contig0_71558	20	19	12	16	13	5	27	11	19	29	22	19	30	30	30	25	23	19
Contig0_71617	22	20	23	20	19	10	22	22	25	29	21	17	28	29	36	28	25	19
Contig0_71647	22	19	17	23	16	13	22	33	22	20	30	18	23	29	35	35	30	25


In [9]:
# Confirm which column I want to use as predictor variable. 
#1= covariate file intercept 
#2 - wet weight (grams, in shell) <-- use this as covariate
#3 - shell length (mm) <-- use this as predictor 

! head ../../data/predictors.size.macau.txt

2.2	17.41
1.9	20.43
2.2	25.33
1.1	19.38
2.2	26.79
1.2	19.8
2.1	20.54
1.9	19.5
1.4	18.43
2.2	21.02


In [10]:
! head ../../data/cov.weight.macau.txt

1	2.2
1	1.9
1	2.2
1	1.1
1	2.2
1	1.2
1	2.1
1	1.9
1	1.4
1	2.2


In [30]:
! head ../2bRAD/HCSSmbdsamples_rab.txt

0	0.001054	0.024934	0.010619	0.005431	0.469934	1e-06	0.096852	0.018074	7e-06	1e-06	0	1e-06	1e-06	0	0	0	1e-06
0.001054	0	0.132783	0.281738	0.178604	0.023215	0.303803	5.1e-05	0.045462	0	1e-06	2.8e-05	3e-06	0	0	0	0.004357	0
0.024934	0.132783	0	0.091518	0.105258	0.010814	0.202782	0.215806	8.6e-05	3.5e-05	9e-06	0.019306	1.3e-05	2e-06	5e-06	2.2e-05	0.032574	6e-06
0.010619	0.281738	0.091518	0	0.178816	0.028384	0.269209	0.003049	0.01418	0	0	3e-06	0	0	0	0	1e-06	0
0.005431	0.178604	0.105258	0.178816	0	1e-06	0.505432	8e-06	0.052672	1e-06	3e-06	0	0	0	0	0	0	0
0.469934	0.023215	0.010814	0.028384	1e-06	0	0	0.098489	0.017889	0	0	2e-06	0	1e-06	1e-06	0	0	0
1e-06	0.303803	0.202782	0.269209	0.505432	0	0	4.1e-05	0.019627	1e-06	0	2e-06	0	0	0	0	1e-06	0
0.096852	5.1e-05	0.215806	0.003049	8e-06	0.098489	4.1e-05	0	0.000844	3e-06	3e-06	1.5e-05	0.00168	0	0.059391	6e-06	0.000188	5e-06
0.018074	0.045462	8.6e-05	0.01418	0.052672	0.017889	0.019627	0.000844	0	0	0	0.000103	1e-06	0	0	0	3e-06	5e-06
7e-06	0	3.5e-05	0	1e-0

Use the following options: 
    
`-g`  specify the methylated read counts file name.  
`-t`  specify the total read counts file name.  
`-p`  specify the predictor variable file name.  
`-n 2` specify which predictor variable column to use in analysis. In our case column 2 (length in mm)  
`-k`  specify the kinship/relatedness matrix file name.  
`-bmm`  specifies binomial mixed model.  
`-o`  specify output file prefix (default “result”).  

Run started at , finished at . 

In [32]:
! /home/shared/macau/macau \
-g ../counts-filtered-meth.txt \
-t ../counts-filtered-total.txt \
-p ../../data/predictors.size.macau.txt \
-n 2 \
-c ../../data/cov.weight.macau.txt \
-k ../2bRAD/HCSSmbdsamples_rab.txt \
-bmm \
-o 20200205-macau

Reading Files ... 
## number of total individuals = 18
## number of analyzed individuals = 18
## number of covariates = 2
## number of total genes/sites = 19648
Performing Analysis ==================================================100.00%


In [33]:
ls

20190812-heatmap-p001.pdf  macau-all-loci.10x75.bed
allLoci.geneinfo.txt       macau-all-loci.5x75.bed
macau.10x75.c.tab          macau-all-loci.bed
macau-10x75perc.bed        macau-any10x.bed
macau.10xall.annot.csv     MACAU.geneinfo.txt
macau.20190812.c.tab       macau-heatmap.10x75.cluster.pdf
macau.5x75.c.tab           macau-heatmap.10x75.length.pdf
macau75.features.csv       macau-heatmap.10x75.tree.pdf
macau75.GO.csv             macau.sign.perc.meth.10x75.bed
macau75.GO.txt             macau.sign.perc.meth.5x75.bed
macau75.sig.csv            output/
macau-all10x.bed


In [34]:
# Peek at the results file 
! head output/20200205-macau.assoc.txt

id	n	acpt_rate	beta	se_beta	pvalue	h	se_h	sigma2	se_sigma2	alpha0	se_alpha0	alpha1	se_alpha1
Contig0_39226	18	4.795e-01	-1.128e-02	3.639e-02	7.565e-01	6.827e-01	2.488e-01	1.236e+00	3.107e+00	8.535e-01	6.794e-01	-1.886e-01	1.683e-01
Contig0_39234	18	3.973e-01	3.811e-02	3.652e-02	2.968e-01	7.570e-01	2.258e-01	1.503e+00	3.325e+00	-1.283e+00	7.052e-01	-1.099e-01	1.626e-01
Contig0_71523	18	2.984e-01	-9.373e-02	7.450e-02	2.083e-01	7.189e-01	2.704e-01	8.320e+00	1.331e+01	3.175e+00	1.366e+00	6.219e-01	3.489e-01
Contig0_71533	18	3.405e-01	1.088e-02	1.502e-01	9.423e-01	3.152e-01	1.767e-01	6.650e+00	5.669e+00	3.392e+00	2.551e+00	9.812e-02	6.823e-01
Contig0_71542	18	3.138e-01	-7.261e-03	9.611e-02	9.398e-01	6.044e-01	2.586e-01	4.163e+00	6.901e+00	4.014e+00	1.740e+00	1.720e-02	5.362e-01
Contig0_71546	18	3.150e-01	-8.721e-02	8.660e-02	3.139e-01	6.522e-01	2.553e-01	4.602e+00	9.124e+00	4.930e+00	1.690e+00	2.095e-01	3.717e-01
Contig0_71558	18	2.732e-01	-6.461e-02	1.233e-01	6.002e-01	5.574e-01	2.850e-01	

In [35]:
# List column names - p-value is 6th column (use for indexing)
! head -n 1 output/20200205-macau.assoc.txt

id	n	acpt_rate	beta	se_beta	pvalue	h	se_h	sigma2	se_sigma2	alpha0	se_alpha0	alpha1	se_alpha1


In [36]:
# Count # hits with p-value <0.05
! awk '$6<0.05{pvalue++} END{print pvalue+0}' \
output/20200205-macau.assoc.txt

1089


In [37]:
# Count # hits with p-value <0.01
! awk '$6<0.01{pvalue++} END{print pvalue+0}' \
output/20200205-macau.assoc.txt

292


In [38]:
# Count # hits with p-value <0.001
! awk '$6<0.001{pvalue++} END{print pvalue+0}' \
output/20200205-macau.assoc.txt

65


In [39]:
# Review run log 
! cat output/20200205-macau.log.txt

##
## MACAU Version = 1.00alpha
##
## Command Line Input = /home/shared/macau/macau -g ../counts-filtered-meth.txt -t ../counts-filtered-total.txt -p ../../data/predictors.size.macau.txt -n 2 -c ../../data/cov.weight.macau.txt -k ../2bRAD/HCSSmbdsamples_rab.txt -bmm -o 20200205-macau 
##
## Date = Fri Feb  7 15:31:45 2020
##
## Summary Statistics:
## number of total individuals = 18
## number of analyzed individuals = 18
## number of total genes/sites = 19648
## number of analyzed genes/sites = 39296
## number of covariates = 2
##
## Computation time:
## total computation time = 20.1332 min 
## computation time break down: 
##      time on eigen-decomposition = 0.5475 min 
##      time on calculating matrix-vector multiplication = 1.64117 min 
##      time on sampling Z = 1.464 min 
##      time on sampling MW = 1.7635 min 
##      time on sampling Hyperparameters = 13.919 min 
##      time on sampling BAU = 1.66033 min 
##
